# 7-3. Getting Started with Real-World Robots - Record your Dataset and Visualize it

:::{note}

This notebook content is derived from lerobot repo's example note "[7_get_started_with_real_robot.md](https://github.com/huggingface/lerobot/blob/main/examples/7_get_started_with_real_robot.md#3-record-your-dataset-and-visualize-it)".

The notebook is formated in Jupyter notebook style, so that one can just follow this notebook on Jetson (running Jupyer Lab server inside the lerobot container).

Please note that this notebook (along with other in the series) may not be the full copy of the original note, as it tries to catpure the essense and important part that involves with Python code executions.<br>
For the full documentation and its update, please always refer to the original document ("[7_get_started_with_real_robot.md](https://github.com/huggingface/lerobot/blob/main/examples/7_get_started_with_real_robot.md)").

:::

## a. Use `koch.yaml` and the `record` function

You can use the `record` function from [`lerobot/scripts/control_robot.py`](../lerobot/scripts/control_robot.py) to achieve efficient data recording. It encompasses many recording utilities:
1. Frames from cameras are saved on disk in threads, and encoded into videos at the end of recording.
2. Video streams from cameras are displayed in window so that you can verify them.
3. Data is stored with [`LeRobotDataset`](../lerobot/common/datasets/lerobot_dataset.py) format which is pushed to your Hugging Face page (unless `--push-to-hub 0` is provided).
4. Checkpoints are done during recording, so if any issue occurs, you can resume recording by re-running the same command again. You can also use `--force-override 1` to start recording from scratch.
5. Set the flow of data recording using command line arguments:
   - `--warmup-time-s` defines the number of seconds before starting data collection. It allows the robot devices to warmup and synchronize (10 seconds by default).
   - `--episode-time-s` defines the number of seconds for data recording for each episode (60 seconds by default).
   - `--reset-time-s` defines the number of seconds for resetting the environment after each episode (60 seconds by default).
   - `--num-episodes` defines the number of episodes to record (50 by default).
6. Control the flow during data recording using keyboard keys:
   - Press right arrow `->` at any time during episode recording to early stop and go to resetting. Same during resetting, to early stop and to go to the next episode recording.
   - Press left arrow `<-` at any time during episode recording or resetting to early stop, cancel the current episode, and re-record it.
   - Press escape `ESC` at any time during episode recording to end the session early and go straight to video encoding and dataset uploading.
7. Similarly to `teleoperate`, you can also use `--robot-path` and `--robot-overrides` to specify your robots.

:::{note}

Before trying `record`, if you want to push your dataset to the hub, make sure you've logged in using a write-access token, which can be generated from the [Hugging Face settings](https://huggingface.co/settings/tokens):
```bash
huggingface-cli login --token ${HUGGINGFACE_TOKEN} --add-to-git-credential
```
Also, store your Hugging Face repository name in a variable (e.g. `cadene` or `lerobot`). For instance, run this to use your Hugging Face user name as repository:
```bash
HF_USER=$(huggingface-cli whoami | head -n 1)
echo $HF_USER
```
If you don't want to push to hub, use `--push-to-hub 0`.

:::

Now run this to record 2 episodes:

In [6]:
!python3 /opt/lerobot/lerobot/scripts/control_robot.py record \
  --robot-path /opt/lerobot/lerobot/configs/robot/koch.yaml \
  --fps 30 \
  --root data \
  --repo-id ${HF_USER}/koch_test \
  --tags tutorial \
  --warmup-time-s 5 \
  --episode-time-s 30 \
  --reset-time-s 30 \
  --num-episodes 2

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.
Error trying to import pynput. Switching to headless mode. As a result, the video stream from the cameras won't be shown, and you won't be able to change the control flow with keyboards. For more info, see traceback below.

Traceback (most recent call last):
  File "/opt/lerobot/lerobot/scripts/control_robot.py", line 225, in is_headless
    import pynput  # noqa
  File "/usr/local/lib/python3.10/dist-packages/pynput/__init__.py", line 40, in <module>
    from . import keyboard
  File "/usr/local/lib/python3.10/dist-packages/pynput/keyboard/__init__.py", line 31, in <module>
    backend = backend(__name__)
  File "/usr/local/lib/python3.10/dist-packages/pynput/_util/__init__.py", lin

## b. Advices for recording dataset

## c. Visualize all episodes
